# Wongnai Review

## Reading and Preprocessing

In [58]:
import pandas as pd

train_df = pd.read_csv("data/wongnai_review/w_review_train.csv", sep=";", header=None).drop_duplicates()
train_df.columns = ["review", "rating"]
test_df = pd.read_csv("data/wongnai_review/test_file.csv", sep=";")

train_df.rating.value_counts() / train_df.shape[0]

4    0.469282
3    0.304328
5    0.169880
2    0.046133
1    0.010377
Name: rating, dtype: float64

In [59]:
two_df = pd.concat([train_df[train_df.rating==2].copy() for i in range(2)]).reset_index(drop=True)
one_df = pd.concat([train_df[train_df.rating==1].copy() for i in range(10)]).reset_index(drop=True)
train_bal = pd.concat([train_df, one_df, two_df]).reset_index(drop=True)
train_bal.rating.value_counts() / df.shape[0]

4    0.392365
3    0.254448
5    0.142036
2    0.115715
1    0.095436
Name: rating, dtype: float64

In [60]:
from tqdm import tqdm_notebook
from pythainlp import word_tokenize
import string, re

def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = "".join(nopunc)
    return [word for word in word_tokenize(nopunc) if word and not re.search(pattern=r"\s+", string=word)]

train_splits = []
test_splits = []
for i in tqdm_notebook(range(train_bal.shape[0])):
    train_splits.append(" ".join(process_text(train_bal["review"][i])))

for i in tqdm_notebook(range(test_df.shape[0])):
    test_splits.append(" ".join(process_text(test_df["review"][i])))

In [61]:
def write_to_file(filename, review, rating=None):
    output = []
    if rating is None:
        output.append("reviewID;review")
        for i in tqdm_notebook(range(len(review))):
            output.append(f"{i + 1};{review[i]}")
    else:
        for i in tqdm_notebook(range(len(review))):
            output.append(f"{review[i]};{rating[i]}")
    with open(filename, "w+") as f:
        f.write("\n".join(output))
    f.close()

write_to_file("data/wongnai_post/w_review_train.csv", train_splits, train_bal["rating"])
write_to_file("data/wongnai_post/test_file.csv", test_splits)

## Training and Prediction

### BERT-Thai

### BERT-Multilingual Cased

## Submission Preperation

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
test_probs = pd.read_csv("output/wongnai_th/test_results.tsv", sep="\t", header=None)
prediction = test_probs.apply(np.argmax, axis=1) + 1

/Users/boom/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [3]:
output = []
output.append("reviewID,rating")
for i in tqdm_notebook(range(len(prediction))):
    output.append(f"{i + 1},{prediction[i]}")
with open("output/wongnai_th/submission.csv", "w+") as f:
    f.write("\n".join(output))
f.close()